In [1]:
import pandas as pd
import numpy as np
import dtale as dt



In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


/home/jeffwa/anaconda3/envs/DL_new/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [3]:
torch.cuda.empty_cache()

In [4]:
y_train=pd.read_pickle('y_train.pkl')
y_test=pd.read_pickle('y_test.pkl')
x_test=pd.read_pickle('x_test.pkl')
x_train=pd.read_pickle('x_train.pkl')

In [5]:
pred_col='log_bullish_week'

In [6]:
test_y=y_test[pred_col]

In [7]:
training_data1=x_train

In [8]:
Y_train=y_train[pred_col]
#Y_train=df['Close']

# Data Loading


In [9]:
from torch.utils.data import Dataset, DataLoader
import joblib
class MyDataset(Dataset):
    def __init__(self, X, Y):
        #normalize

    
        self.X = torch.tensor(X, dtype=torch.float32)
        new_shape = (len(Y), 1)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.Y = self.Y.view(new_shape)
        
    
        
        #self.Y = self.Y.view(new_shape)
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# Normalize the X data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [10]:
#scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = StandardScaler()
scaled_fit=scaler.fit_transform(training_data1)
        

joblib.dump(scaler, 'sc.joblib') 

['sc.joblib']

In [11]:
#scaler_y = StandardScaler()
scaled_fit_y=Y_train.values.reshape(-1, 1)
       

In [12]:
# Assuming X is a DataFrame with 5 columns and Y is a DataFrame with 1 column
#train_data = MyDataset(scaled_fit, Y_train.values.reshape(-1, 1))
train_data = MyDataset(scaled_fit, scaled_fit_y)

train_loader = DataLoader(train_data, shuffle=True, batch_size=32) #

In [13]:
training_data1

,Heiken Ashi Low,Heiken Ashi Close,log_normalized_change,log_price_range,log_premarket_changes,log_Smart_Money,log_volume_deviation,log_norm_avg_deviation_200,log_norm_avg_deviation_300,log_ha_change
299,110.269111,110.852270,0.000199,0.002583,-0.000265,0.000464,-0.441282,0.091401,0.120446,0.001375
300,110.242772,110.584226,-0.001326,0.005248,-0.002454,0.001128,-0.196779,0.089073,0.118519,-0.002421
301,110.419053,111.063419,0.002717,0.016162,-0.000729,0.003446,0.040220,0.090741,0.120595,0.004324
302,109.611300,110.257496,-0.010508,0.011656,-0.002849,-0.007659,0.427762,0.079324,0.109514,-0.007283
303,109.941750,110.378666,0.008521,0.008049,0.001136,0.007385,0.098847,0.086907,0.117429,0.001098
...,...,...,...,...,...,...,...,...,...,...
4262,402.350006,408.007500,0.010572,0.027746,-0.003129,0.013701,0.231950,0.047874,0.004319,0.010545
4263,405.563838,415.707497,0.014452,0.013066,0.009835,0.004617,0.223718,0.062525,0.019111,0.018696
4264,410.635667,413.000000,-0.010686,0.014202,-0.012531,0.001845,0.125602,0.052183,0.008807,-0.006534
4265,408.100006,409.752502,-0.006130,0.007786,-0.006228,0.000098,-0.323262,0.046425,0.003089,-0.007894


# Pytorch setup

In [14]:
# Define your LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(LSTM,self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Reshape the input tensor to match the expected shape of (batch_size, sequence_length, input_size)
        x = x.view(-1, 1, x.shape[-1])
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, (h_n, c_n) = self.lstm(x)
        # Apply activation function
        output = self.relu(output)
        output = self.dropout(output[:, -1, :])
        
        output = self.fc(output)
        output = self.sigmoid(output)

        return torch.squeeze(output)

In [15]:
num_epochs = 50200
learning_rate = 0.0005
num_layers=5
input_size = 10
hidden_size = 90
output_size = 1
dropout=.5

In [16]:
#set up gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# Instantiate the model
model = LSTM(input_size=input_size, hidden_size=hidden_size,num_layers=num_layers,output_size=1,dropout=dropout)#output_size=1

model.to(device)
train_data_x=train_data.X.to(device)
train_data_y=train_data.Y.to(device)
# Define the loss function and optimizer
criterion = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.003,weight_decay=6e-7) 


In [18]:
# Train the model
for epoch in range(num_epochs):
    outputs = model(train_data_x)
    loss = criterion(outputs, train_data_y.squeeze())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
    """for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y.squeeze())
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))"""

Epoch: 0, loss: 0.72633
Epoch: 100, loss: 0.37494
Epoch: 200, loss: 0.34791
Epoch: 300, loss: 0.30058
Epoch: 400, loss: 0.25165
Epoch: 500, loss: 0.22959
Epoch: 600, loss: 0.20974
Epoch: 700, loss: 0.18964
Epoch: 800, loss: 0.19281
Epoch: 900, loss: 0.29543
Epoch: 1000, loss: 0.20863
Epoch: 1100, loss: 0.17710
Epoch: 1200, loss: 0.17765
Epoch: 1300, loss: 0.23975
Epoch: 1400, loss: 0.20177
Epoch: 1500, loss: 0.11599
Epoch: 1600, loss: 0.12145
Epoch: 1700, loss: 0.20327
Epoch: 1800, loss: 0.12347
Epoch: 1900, loss: 0.10357
Epoch: 2000, loss: 0.07243
Epoch: 2100, loss: 0.06682
Epoch: 2200, loss: 0.04302
Epoch: 2300, loss: 0.02767
Epoch: 2400, loss: 0.01960
Epoch: 2500, loss: 0.01230
Epoch: 2600, loss: 0.00918
Epoch: 2700, loss: 0.00912
Epoch: 2800, loss: 0.00234
Epoch: 2900, loss: 0.00129
Epoch: 3000, loss: 0.00140
Epoch: 3100, loss: 0.00067
Epoch: 3200, loss: 0.00079
Epoch: 3300, loss: 0.00045
Epoch: 3400, loss: 0.00059
Epoch: 3500, loss: 0.00046
Epoch: 3600, loss: 0.00053
Epoch: 3700, 

Epoch: 29700, loss: 0.00001
Epoch: 29800, loss: 0.00001
Epoch: 29900, loss: 0.00001
Epoch: 30000, loss: 0.00001
Epoch: 30100, loss: 0.00001
Epoch: 30200, loss: 0.00001
Epoch: 30300, loss: 0.00051
Epoch: 30400, loss: 0.00011
Epoch: 30500, loss: 0.00002
Epoch: 30600, loss: 0.00001
Epoch: 30700, loss: 0.00001
Epoch: 30800, loss: 0.00001
Epoch: 30900, loss: 0.00001
Epoch: 31000, loss: 0.00001
Epoch: 31100, loss: 0.00001
Epoch: 31200, loss: 0.00001
Epoch: 31300, loss: 0.00001
Epoch: 31400, loss: 0.00001
Epoch: 31500, loss: 0.00001
Epoch: 31600, loss: 0.00001
Epoch: 31700, loss: 0.00001
Epoch: 31800, loss: 0.00002
Epoch: 31900, loss: 0.00001
Epoch: 32000, loss: 0.00004
Epoch: 32100, loss: 0.00001
Epoch: 32200, loss: 0.00001
Epoch: 32300, loss: 0.00001
Epoch: 32400, loss: 0.00001
Epoch: 32500, loss: 0.00001
Epoch: 32600, loss: 0.00001
Epoch: 32700, loss: 0.00493
Epoch: 32800, loss: 0.00006
Epoch: 32900, loss: 0.00002
Epoch: 33000, loss: 0.00003
Epoch: 33100, loss: 0.00002
Epoch: 33200, loss: 

In [19]:
# evaluate the network multiple times with dropout turned on
#model.train()



In [20]:
sc = joblib.load('sc.joblib')
#transformedData = sc.transform(test_x)
transformedData = sc.transform(x_test)

In [21]:
#test_y=Y_train[-128:].values.reshape(-1, 1)
transformedData_y=test_y.values.reshape(-1, 1)

In [22]:
# Use the trained model for predictions
test_data = MyDataset(transformedData, transformedData_y)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [23]:
model.eval()

LSTM(
  (lstm): LSTM(10, 90, num_layers=5, batch_first=True)
  (fc): Linear(in_features=90, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [24]:
test_data_x=test_data.X.to(device)

#====================
#testing purposes only
#test_data_x=train_data.X[-21:].to(device)
#test_data.Y.to(device)

In [25]:
outputlist=[]
model.train()
len_outputs=100
with torch.no_grad():
    for i in range(len_outputs):
        output = model(test_data_x)
        outputlist.append(output)
model.eval()

LSTM(
  (lstm): LSTM(10, 90, num_layers=5, batch_first=True)
  (fc): Linear(in_features=90, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [26]:
scaled_output_list=[]
for outputs in outputlist:
    scaled_output=outputs.cpu().reshape(-1, 1)
    scaled_output_list.append(scaled_output)

In [27]:
def array_list_to_dataframe(arrays,length):
    # Convert the list of arrays to a single 2D array
    array_2d = np.concatenate(arrays, axis=1)
    
    # Reshape the 2D array into a 5-column array
    array_5col = np.reshape(array_2d, (-1, length))
    
    # Create a DataFrame from the 5-column array
    df = pd.DataFrame(array_5col)
    
    return df

In [52]:
array_list_to_dataframe(scaled_output_list,len_outputs )> .9999

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
test=array_list_to_dataframe(scaled_output_list,len_outputs )
testarray=test.iloc[:,:]

# Sigmoid Label Probability

In [75]:
count1 = (testarray > .99).sum(axis=1)
count2=(testarray > .999).sum(axis=1)
count3=(testarray > .9999).sum(axis=1)

In [78]:
count3

0     0
1     0
2     0
3     0
4     0
5     0
6     4
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [68]:
test

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,2.677959e-04,1.869752e-07,1.324246e-05,5.404076e-05,4.187386e-05,2.107408e-03,4.521722e-07,9.557757e-07,2.716579e-05,2.368615e-03,...,5.070763e-05,1.494888e-05,4.365377e-06,1.664984e-06,5.033584e-06,5.017869e-07,8.589566e-05,1.646490e-06,2.209916e-05,5.278027e-06
1,6.864561e-08,7.567501e-11,5.358310e-10,1.019591e-07,2.116911e-08,8.415214e-11,2.130656e-11,1.668469e-07,3.446665e-09,5.527184e-08,...,9.612493e-11,1.823665e-09,1.298457e-07,1.299509e-13,1.978204e-09,1.428204e-10,3.173640e-10,3.577991e-11,7.667950e-13,1.774638e-08
2,3.744739e-02,3.128206e-04,6.454992e-03,8.355989e-02,7.640725e-03,3.376525e-04,2.231725e-03,1.062144e-02,1.329030e-03,4.027835e-02,...,1.003459e-01,6.768250e-03,8.658184e-05,5.096021e-03,8.325430e-04,1.518515e-04,7.795495e-04,6.969220e-04,1.159364e-02,3.167674e-03
3,5.983695e-05,6.833614e-05,3.431430e-06,1.225967e-03,2.810460e-06,3.292839e-04,3.224208e-04,5.457075e-06,6.393307e-05,1.313614e-05,...,5.033896e-06,7.016864e-05,8.195769e-05,2.871876e-04,1.582606e-03,6.112871e-06,1.748495e-03,6.863639e-06,6.345444e-05,3.660891e-06
4,2.298653e-11,1.934863e-10,3.413874e-08,8.658160e-12,1.357800e-07,5.578721e-08,3.044452e-12,4.873286e-12,1.365987e-09,7.387678e-14,...,1.099348e-10,4.669273e-09,8.439948e-08,2.736210e-13,8.839605e-13,2.880191e-09,5.893885e-13,7.661763e-10,2.939382e-11,1.386001e-09
5,2.013646e-02,8.996826e-02,6.359720e-01,5.163308e-01,6.859729e-01,5.761038e-01,2.951187e-01,7.719985e-01,4.919725e-01,3.711081e-01,...,3.790920e-01,7.865450e-01,6.082415e-01,9.563299e-01,7.545440e-01,1.544015e-01,4.007889e-01,8.791486e-01,9.462767e-01,5.881965e-01
6,9.970293e-01,9.997557e-01,9.990820e-01,9.987380e-01,9.995341e-01,9.995458e-01,9.997081e-01,9.954786e-01,9.996848e-01,9.977561e-01,...,9.993502e-01,9.863589e-01,9.998575e-01,9.995059e-01,9.911534e-01,9.998210e-01,9.994030e-01,9.987005e-01,9.990639e-01,9.955074e-01
7,9.527294e-06,1.458096e-05,8.292510e-06,8.911487e-06,2.487943e-06,1.045544e-04,7.070566e-06,3.310117e-06,4.961858e-05,1.136552e-06,...,4.983812e-06,7.183745e-05,1.009643e-05,3.333535e-06,1.111876e-05,3.282698e-06,4.109027e-05,1.336039e-05,5.356836e-03,1.237875e-05
8,3.634983e-04,2.243139e-05,2.546468e-03,1.010035e-03,4.846599e-04,2.451232e-05,1.366769e-03,2.381715e-04,8.968060e-04,7.068974e-05,...,2.157140e-05,2.080366e-02,7.041308e-02,3.954768e-04,1.004166e-02,1.147873e-03,2.722435e-03,1.103688e-02,9.603875e-05,2.361841e-03
9,4.026591e-10,1.424863e-09,6.754451e-08,1.467963e-07,1.611446e-07,4.240135e-08,1.866329e-08,3.012423e-08,8.379122e-09,2.533608e-10,...,6.438805e-06,1.681611e-06,6.317435e-11,1.801661e-06,6.062972e-08,8.498639e-11,7.864494e-09,7.213227e-07,2.550978e-06,1.103231e-09


In [32]:
torch.cuda.empty_cache()

In [33]:
#13000 dropout .5
#loss .02797

In [34]:
#13000 dropout .5
#loss: 0.02298
#loss .0195 lowest